<a href="https://colab.research.google.com/github/Shamil2007/DataAnalysis/blob/main/Data-Cleaning/handling_dublicated_incorrectDataTypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = [
    {'UserID':1001, 'Email':'alex@corp.com', 'Signup_Date':'2023-01-05',
     'Price':'$1,200.00', 'Department':'Sales', 'Purchase_Amount':'100.50'},
    {'UserID':1002, 'Email':'beth@corp.com', 'Signup_Date':'01/07/2023',
     'Price':'$350', 'Department':'marketing', 'Purchase_Amount':'200'},

    {'UserID':1001, 'Email':'alex@corp.com', 'Signup_Date':'2023-03-10',
     'Price':'$1,250.00', 'Department':'Sales', 'Purchase_Amount':'150.25'},

    {'UserID':1003, 'Email':'carl@corp.com', 'Signup_Date':'2023-02-15',
     'Price':'$99.99', 'Department':'engineering', 'Purchase_Amount':'50'},

    {'UserID':1004, 'Email':'dana@corp.com', 'Signup_Date':'2023-02-16',
     'Price':'$1,000', 'Department':'Sales', 'Purchase_Amount':'75'},
    {'UserID':1004, 'Email':'dana@corp.com', 'Signup_Date':'2023-02-16',
     'Price':'$1,000', 'Department':'Sales', 'Purchase_Amount':'75'},

    {'UserID':1005, 'Email':'erin@corp.com', 'Signup_Date':'15-03-2023',
     'Price':'$2,500.50', 'Department':'Marketing', 'Purchase_Amount':'300'},
    {'UserID':1006, 'Email':'frank@corp.com', 'Signup_Date':'2023/03/20',
     'Price':'$0', 'Department':'Support', 'Purchase_Amount':'80'},
    {'UserID':1007, 'Email':'gina@corp.com', 'Signup_Date':'2023-04-01',
     'Price':'—', 'Department':'Support', 'Purchase_Amount':'NaN'},
    {'UserID':1008, 'Email':'hank@corp.com', 'Signup_Date':'2023-03-25',
     'Price':'$0', 'Department':'Engineering', 'Purchase_Amount':'0'},

    {'UserID':1002, 'Email':'beth@corp.com', 'Signup_Date':'2023-07-02',
     'Price':'$355', 'Department':'Marketing', 'Purchase_Amount':'220'},

    {'UserID':1009, 'Email':' ivy@corp.com ', 'Signup_Date':'2023-07-10',
     'Price':'$1,100', 'Department':' sales ', 'Purchase_Amount':'120.00'},
]

df = pd.DataFrame(data)


#Part 1 — Deep Dive on Duplicates

##1.1 What Are Duplicates?

1. True duplicates: Entire rows are identical across all columns (bit-for-bit the same).

2. Partial duplicates: Rows share the same value on a key subset of columns (e.g., UserID, Email) but differ elsewhere (e.g., timestamps, amounts). These typically represent the same entity recorded multiple times.

Analogy: True duplicates are identical photocopies. Partial duplicates are the same person’s record in two systems with slightly different details.

##1.2 Why They Occur (Root Causes)

1. Data entry errors (double submissions, form resubmits)

2. Multi-source integration without proper keys or standardization

3. ETL glitches (bad joins, reprocessing a batch)

4. Application bugs (duplicate writes / retries without idempotency)

##1.3 Why They Are Problematic

Skews analysis: Inflated counts, distorted averages (e.g., double-counted orders increase revenue and LTV).

Biases ML models: The same pattern “learned twice” → overfitting → poor generalization.

Wastes compute: Bigger datasets with no added information.

Bad business decisions: Leadership acts on inaccurate totals.

##1.4 How to Identify Duplicates

In [3]:
# Count true duplicates (all columns)
df.duplicated().sum()

np.int64(1)

In [4]:
df

,UserID,Email,Signup_Date,Price,Department,Purchase_Amount
0,1001,alex@corp.com,2023-01-05,"$1,200.00",Sales,100.50
1,1002,beth@corp.com,01/07/2023,$350,marketing,200
2,1001,alex@corp.com,2023-03-10,"$1,250.00",Sales,150.25
3,1003,carl@corp.com,2023-02-15,$99.99,engineering,50
4,1004,dana@corp.com,2023-02-16,"$1,000",Sales,75
5,1004,dana@corp.com,2023-02-16,"$1,000",Sales,75
6,1005,erin@corp.com,15-03-2023,"$2,500.50",Marketing,300
7,1006,frank@corp.com,2023/03/20,$0,Support,80
8,1007,gina@corp.com,2023-04-01,—,Support,NaN
9,1008,hank@corp.com,2023-03-25,$0,Engineering,0


In [5]:
# Inspect true duplicates
df[df.duplicated(keep=False)].sort_values(list(df.columns)).head()

,UserID,Email,Signup_Date,Price,Department,Purchase_Amount
4,1004,dana@corp.com,2023-02-16,"$1,000",Sales,75
5,1004,dana@corp.com,2023-02-16,"$1,000",Sales,75


In [6]:
df.duplicated(subset=['UserID']).sum()

np.int64(3)

In [7]:
df[df.duplicated(subset=['UserID'], keep=False)].sort_values('UserID').head()

,UserID,Email,Signup_Date,Price,Department,Purchase_Amount
0,1001,alex@corp.com,2023-01-05,"$1,200.00",Sales,100.50
2,1001,alex@corp.com,2023-03-10,"$1,250.00",Sales,150.25
1,1002,beth@corp.com,01/07/2023,$350,marketing,200
10,1002,beth@corp.com,2023-07-02,$355,Marketing,220
4,1004,dana@corp.com,2023-02-16,"$1,000",Sales,75


###The `keep` parameter is crucial:
- keep = 'first' (default): Marks duplicates as True, except for the first occurrence.
- keep = 'last': Marks duplicates as True, except for the last occurrence.
- keep = False: Marks all duplicates as True. This is best for identifying all rows involved.

In [8]:
df['UserID'].value_counts().head()

,count
UserID,
1001,2
1002,2
1004,2
1003,1
1005,1


In [9]:
for user_id, count in df['UserID'].value_counts().items():
    if count >= 2:
        print(f"ID: {user_id} Count: {count}")

ID: 1001 Count: 2
ID: 1002 Count: 2
ID: 1004 Count: 2


##1.5 How to Handle Duplicates (Decision Framework)

Rule #1: Investigate before you delete.
Ask: Are these artifacts or legitimate multi-records (e.g., multiple orders by same user)? Clarify the business rule.

###Common Strategies

1. Remove true duplicates (safe and typical):

In [10]:
# Drop exact row duplicates
df = df.drop_duplicates(keep='first', ignore_index=True)
df

,UserID,Email,Signup_Date,Price,Department,Purchase_Amount
0,1001,alex@corp.com,2023-01-05,"$1,200.00",Sales,100.50
1,1002,beth@corp.com,01/07/2023,$350,marketing,200
2,1001,alex@corp.com,2023-03-10,"$1,250.00",Sales,150.25
3,1003,carl@corp.com,2023-02-15,$99.99,engineering,50
4,1004,dana@corp.com,2023-02-16,"$1,000",Sales,75
5,1005,erin@corp.com,15-03-2023,"$2,500.50",Marketing,300
6,1006,frank@corp.com,2023/03/20,$0,Support,80
7,1007,gina@corp.com,2023-04-01,—,Support,NaN
8,1008,hank@corp.com,2023-03-25,$0,Engineering,0
9,1002,beth@corp.com,2023-07-02,$355,Marketing,220


In [11]:
# Assumes you’ve parsed dates; keep latest per UserID
df = df.sort_values(['UserID', 'Signup_Date'])
df = df.drop_duplicates(subset=['UserID'], keep='last', ignore_index=True)
df

,UserID,Email,Signup_Date,Price,Department,Purchase_Amount
0,1001,alex@corp.com,2023-03-10,"$1,250.00",Sales,150.25
1,1002,beth@corp.com,2023-07-02,$355,Marketing,220
2,1003,carl@corp.com,2023-02-15,$99.99,engineering,50
3,1004,dana@corp.com,2023-02-16,"$1,000",Sales,75
4,1005,erin@corp.com,15-03-2023,"$2,500.50",Marketing,300
5,1006,frank@corp.com,2023/03/20,$0,Support,80
6,1007,gina@corp.com,2023-04-01,—,Support,NaN
7,1008,hank@corp.com,2023-03-25,$0,Engineering,0
8,1009,ivy@corp.com,2023-07-10,"$1,100",sales,120.00


#Part 2 — Deep Dive on Incorrect Data Types

##2.1 What Are Data Types?

###Common pandas dtypes:

- object (usually strings)

- int32 / float32 (numeric)

- datetime64[ns] (timestamps)

- bool

- category (efficient for repeated labels)

##2.2 Why Incorrect Types Occur

- Numbers stored as strings (currency symbols $, commas ,, hidden spaces)

- Dates stored as strings with mixed formats

- Categorical text left as object

- CSV/Excel import quirks (inferred types wrong)

##2.3 Why It’s a Problem

- No math on strings (sum/avg fail or coerce).

- Wrong sorting: "1" < "10" < "2" (alphabetical, not numeric).

- Memory bloat: object wastes RAM; category is compact.

Library errors: ML needs numeric; time series needs datetime.

##2.4 How to Identify Types

In [12]:
df.dtypes

,0
UserID,int64
Email,object
Signup_Date,object
Price,object
Department,object
Purchase_Amount,object


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   UserID           9 non-null      int64 
 1   Email            9 non-null      object
 2   Signup_Date      9 non-null      object
 3   Price            9 non-null      object
 4   Department       9 non-null      object
 5   Purchase_Amount  9 non-null      object
dtypes: int64(1), object(5)
memory usage: 564.0+ bytes


In [14]:
df.head()

,UserID,Email,Signup_Date,Price,Department,Purchase_Amount
0,1001,alex@corp.com,2023-03-10,"$1,250.00",Sales,150.25
1,1002,beth@corp.com,2023-07-02,$355,Marketing,220
2,1003,carl@corp.com,2023-02-15,$99.99,engineering,50
3,1004,dana@corp.com,2023-02-16,"$1,000",Sales,75
4,1005,erin@corp.com,15-03-2023,"$2,500.50",Marketing,300


In [15]:
# Strip currency symbols, commas, stray spaces, and convert
df['Price'] = (df['Price']
    .astype(str)
    .str.replace(r'[^0-9\.\-]', '', regex=True)  # keep digits, dot, minus
    .str.strip()
    .replace('', pd.NA)
)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df.head()

,UserID,Email,Signup_Date,Price,Department,Purchase_Amount
0,1001,alex@corp.com,2023-03-10,1250.00,Sales,150.25
1,1002,beth@corp.com,2023-07-02,355.00,Marketing,220
2,1003,carl@corp.com,2023-02-15,99.99,engineering,50
3,1004,dana@corp.com,2023-02-16,1000.00,Sales,75
4,1005,erin@corp.com,15-03-2023,2500.50,Marketing,300


Why errors='coerce'? Turns unparseable junk into NaN so you can see and handle it explicitly.

In [16]:
# Mixed formats: let pandas parse, preserve failures as NaT
df['Signup_Date'] = pd.to_datetime(
    df['Signup_Date'], errors='coerce', dayfirst=True
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   UserID           9 non-null      int64         
 1   Email            9 non-null      object        
 2   Signup_Date      4 non-null      datetime64[ns]
 3   Price            8 non-null      float64       
 4   Department       9 non-null      object        
 5   Purchase_Amount  9 non-null      object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 564.0+ bytes


In [17]:
df['Department'].value_counts()

,count
Department,
Sales,2
Marketing,2
Support,2
engineering,1
Engineering,1
sales,1


In [18]:
df['Department'] = df['Department'].str.strip().str.title().astype('category')
df['Department'].value_counts()

,count
Department,
Sales,3
Engineering,2
Marketing,2
Support,2


In [20]:
df['Purchase_Amount'] = df['Purchase_Amount'].astype(np.float32)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   UserID           9 non-null      int64         
 1   Email            9 non-null      object        
 2   Signup_Date      4 non-null      datetime64[ns]
 3   Price            8 non-null      float64       
 4   Department       9 non-null      category      
 5   Purchase_Amount  8 non-null      float32       
dtypes: category(1), datetime64[ns](1), float32(1), float64(1), int64(1), object(1)
memory usage: 669.0+ bytes
